#### -- [INIT] -- 
***

##### **Import Libraries and Init Globals**
- `on_awake   ` (init_game => screen, refresh, load assets ... etc)
- `on_exit    ` (close_app => clear memory and shut down cpu/sys ... etc)

In [29]:
import pygame
from typing import NamedTuple, Tuple, List
import os
import sys

class SystemContext(NamedTuple):
    screen :pygame.Surface
    clock :pygame.time.Clock

# System Context will be init on awake
class RenderContext(NamedTuple):
    WIDTH :int
    HEIGHT :int
    BACKGROUND_COLOR :tuple
    FPS :int

class AssetContext(NamedTuple):
    ASSET_FOLDER_PATH :str

# Statically define CONTEXT CONSTANTS
RENDER_CONTEXT = RenderContext(
    640, # RESOLUTION_WIDTH
    480, # RESOLUTION_HEIGHT
    (255, 255, 0), # BACKGROUND_COLOR
    5 # FPS
)

ASSET_CONTEXT = AssetContext(
    ASSET_FOLDER_PATH = os.path.join('assets','images')
)

def on_awake()-> SystemContext:
    ''' 
    Initialize pygame
    - SYS_SCREEN
    - SYS_CLOCK
    '''
    # Initialize pygame
    pygame.init()
    # Initialize the screen
    screen = pygame.display.set_mode(
        # @audit-ok 💨 : Use `HWSURFACE` and `DOUBLEBUF` for better performance
        (RENDER_CONTEXT.WIDTH, RENDER_CONTEXT.HEIGHT),
        pygame.HWSURFACE | pygame.DOUBLEBUF)
    # Initialize the clock
    # @audit-ok 💨 : Cap the frame rate else loop will run at system allows and 
    # # will lead to :
    # - high CPU usage
    # - inconsistent performance across different devices. 
    clock = pygame.time.Clock()

    # Set the title and icon (optional)
    pygame.display.set_caption('Asteroids_ECS_Edition')
    icon = pygame.image.load(
        os.path.join(ASSET_CONTEXT.ASSET_FOLDER_PATH, 'app_icon.jpg')
    ).convert_alpha()
    pygame.display.set_icon(icon)

    return SystemContext(screen, clock)

def on_exit() -> None :
    ''' 
    Handle exit event
    '''
    pygame.quit()
    sys.exit()

#### -- [ENTITY] --
***

In [1]:
import uuid

# Implement a simple Entity class that manages a unique identifier for each entity
class Entity:
    def __init__(self, id=None):
        self.id = id or self.generate_unique_id()

    @staticmethod
    def generate_unique_id():
        return str(uuid.uuid4())

#### -- [COMPONENT] --
***

Component Base

In [41]:
# Create component classes
# These classes should ONLY contain data, NO METHODS

# transforms
class Position:
    def __init__(self, x, y):
        self.x = x
        self.y = y

# state
class Health:
    def __init__(self, health):
        self.health = health

# graphics
class Shape:
    def __init__(self, points:List[Tuple[float,float]]):
        self.points = points

Component Manager

In [3]:
from abc import ABC, abstractmethod
from collections import defaultdict

class ComponentManagerInterface(ABC):
    @abstractmethod
    def add_component(self, entity, component):
        'Add a component to an entity'
        pass

    @abstractmethod
    def remove_component(self, entity, component_type):
        'Remove a component of a specific type from an entity'
        pass

    @abstractmethod
    def get_component(self, entity, component_type):
        'Get a component of a specific type from an entity'
        pass

    @abstractmethod
    def get_entities_with_component(self, component_type):
        'Get a list of entities that have a component of a specific type'
        pass

# Implement a simple ComponentManager class that manages a dictionary of components
class ComponentManager(ComponentManagerInterface):
    def __init__(self):
        # With a `defaultdict`, you can directly add a component without first 
        # checking if the `entity.id` exists in `self.components`. 
        # If the `entity.id` is not present, a new dictionary will be created 
        # automatically :
        # - simplifies the code for `add_component`
        self.components = defaultdict(dict)

    def add_component(self, entity, component):
        # defaultdict will automatically create a new dictionary for the entity
        # if it does not exist : greatly simplifies the code
        self.components[entity.id][type(component)] = component

    def remove_component(self, entity, component_type):
        if entity.id in self.components and component_type in self.components[entity.id]:
            del self.components[entity.id][component_type]

    def get_component(self, entity, component_type):
        return self.components[entity.id].get(component_type, None)

    def get_entities_with_component(self, component_type):
        entities = []
        for entity_id, components in self.components.items():
            if component_type in components:
                entities.append(Entity(entity_id))
        return entities


#### -- [ASSETS] --

Generate

In [36]:
import math
from numpy import random

def generate_asteroid_points(size, num_points)-> List[Tuple[float,float]]:
    points = []
    angle = 2 * math.pi/num_points

    for i in range(num_points):
        length = random.randint(int(size * 0.5), size)
        x = length * math.cos(angle * i)
        y = length * math.sin(angle * i)
        points.append((x,y))

    return points

#### -- [SYSTEM]--
***


In [42]:
from numpy import random
# Implement Systems that operate on the components
class MovementSystem:
    def update(self, component_manager, delta_time):
        entities = component_manager.get_entities_with_component(Position)
        for entity in entities:
            position = component_manager.get_component(entity, Position)
            position.x += 1 * delta_time
            position.y += 1 * delta_time

class RenderSystem:
    def __init__(self, screen :pygame.Surface, clock :pygame.time.Clock):
        self.screen = screen
        self.clock = clock

    def update(self, component_manager):
        # Fill the screen with the background color
        rgb_random = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
        # self.screen.fill(RENDER_CONTEXT.BACKGROUND_COLOR)
        self.screen.fill(rgb_random)

        # Render all entities with a position and points component
        entities = component_manager.get_entities_with_component(Shape)
        for entity in entities:
            position = component_manager.get_component(entity, Position)
            if(position is None): 
                continue
            x,y = position.x, position.y
            shape = component_manager.get_component(entity, Shape)
            pygame.draw.polygon(
            self.screen, 
            (255, 255, 255), 
            [(x+px, y+py) for px,py in shape.points], 
            1
        )

        # Update the display
        pygame.display.update()

        # Limit the frame rate
        self.clock.tick(RENDER_CONTEXT.FPS)


#### -- [RUNTIME] --
***

In [44]:
# Initialization
context_sys = on_awake()
component_manager = ComponentManager()
movement_system = MovementSystem()
render_system = RenderSystem(context_sys.screen, context_sys.clock)

# Create entities and components
asteroid = Entity()
asteroid_position = Position(320, 240)
asteroid_health = Health(100)
asteroid_points = generate_asteroid_points(100, 10)
asteroid_shape = Shape(asteroid_points)

component_manager.add_component(asteroid, asteroid_position)
component_manager.add_component(asteroid, asteroid_health)
component_manager.add_component(asteroid, asteroid_shape)

# Main game loop
while True:
    try:
        # Set the frame rate
        context_sys.clock.tick(RENDER_CONTEXT.FPS)
        # Event Handling
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                on_exit()
            if event.type == pygame.KEYUP:
                if event.key == pygame.K_ESCAPE:
                    on_exit()
        # Update inputs
        # Update positions
        movement_system.update(component_manager, 0.016)
        # Update the display
        render_system.update(component_manager)
            
    except SystemExit:
        break

#### -- [FOOTNOTES] --
***

Explain `type checking` in python
***

Some key points:

1. Type hints only `optionally enforce types` - you need a separate type  
checking tool. But they `clearly document the expected types` for your function.

2. You can specify types for all function arguments and the return value.

3. Use List[type] to denote a list of elements of type type.

4. Use Tuple[type1, type2] to denote a tuple with two elements of types type1 and type2.

5. You can use int, str, float, bool, etc. to specify basic types. And List,  
Tuple, Dict to specify compound types.

6. Type hints `do not affect runtime behavior` - they only provide `static`  
type checks.

7. You can use a tool like `mypy` to actually `run the type checks and enforce`  
the hints. 

8. Specifying strict type hints makes your code more robust, documented and  
catch bugs early. 

You can use type checking to ensure that a function only accepts a list of  
points (x, y coordinates). You can do it like this:

```python
from typing import List, Tuple

def function_name(points: List[Tuple[int, int]]) -> None:
  """Only accepts a list of (x, y) point tuples"""
  # Function logic here 
```

This uses Type Hints to specify that the points argument must be a List of  
Tuples, where each Tuple has two int elements. 
So these would be valid calls:

```python
function_name([(1, 2), (3, 4)]) 
function_name([[1, 2], [3, 4]])  # Also ok, treats each sublist as a point
```

But these would throw a TypeError:

```python
function_name((1, 2, 3))  # Not a list 
function_name([(1, '2')])  # Not ints 
function_name(['string', 'list'])  # Not tuples
```